In [ ]:
from collections import defaultdict
import os
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_colwidth', 500)

s3 = S3AccessUtils('/root/data')
RDS = RDSAccessUtils()

<h1> Rectification Library </h1>

In [ ]:
import json
from typing import Dict
import cv2
import numpy as np


IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000


def get_camera_parameters(params: Dict) -> Dict:
    """Return individual camera parameters from JSON stereo parameters contents."""

    camera_matrix_1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    camera_matrix_2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    dist_coeffs_1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                    params['CameraParameters1']['TangentialDistortion'] + \
                    [params['CameraParameters1']['RadialDistortion'][2]]
    dist_coeffs_1 = np.array(dist_coeffs_1)

    dist_coeffs_2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                    params['CameraParameters2']['TangentialDistortion'] + \
                    [params['CameraParameters2']['RadialDistortion'][2]]
    dist_coeffs_2 = np.array(dist_coeffs_2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(camera_matrix_1, dist_coeffs_1,
                                                               camera_matrix_2,
                                                               dist_coeffs_2, image_size, R, T,
                                                               None,
                                                               None,
                                                               None, None, None,
                                                               cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(camera_matrix_1, dist_coeffs_1, R1, P1, image_size,
                                            cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(camera_matrix_2, dist_coeffs_2, R2, P2, image_size,
                                             cv2.CV_16SC2)

    params = {
        'left_maps': left_maps,
        'right_maps': right_maps,
        'camera_matrix_1': camera_matrix_1,
        'dist_coeffs_1': dist_coeffs_1,
        'R1': R1,
        'P1': P1,
        'camera_matrix_2': camera_matrix_2,
        'dist_coeffs_2': dist_coeffs_2,
        'R2': R2,
        'P2': P2
    }
    return params


def unrectify(ann: Dict, params: Dict):
    """Un-rectify ann with params."""

    left_maps = params['left_maps']
    right_maps = params['right_maps']

    ann_u = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                y_new, x_new = left_maps[0][y, x]
            elif side == 'rightCrop':
                y_new, x_new = right_maps[0][y, x]
            else:
                raise Exception('Invalid side!')

            ann_u[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_u


def rectify(ann: Dict, params: Dict) -> Dict:
    """Rectify ann with params."""

    camera_matrix_1 = params['camera_matrix_1']
    dist_coeffs_1 = params['dist_coeffs_1']
    R1 = params['R1']
    P1 = params['P1']

    camera_matrix_2 = params['camera_matrix_2']
    dist_coeffs_2 = params['dist_coeffs_2']
    R2 = params['R2']
    P2 = params['P2']

    ann_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[y, x]]).astype(float),
                        camera_matrix_1,
                        dist_coeffs_1,
                        R=R1,
                        P=P1)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[y, x]]).astype(float),
                        camera_matrix_2,
                        dist_coeffs_2,
                        R=R2,
                        P=P2)[0][0]
            else:
                raise Exception('Invalid side!')

            ann_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_r


def un_re_rectify(df, params_o, params_n):

    ann_u_rs = []
    for idx, row in df.iterrows():
        ann = row.annotation
        if ann is None:
            ann_u_rs.append(None)
            continue

        # un-rectify with original params
        ann_u = unrectify(ann, params_o)
        if ann_u:

            # re-rectify with new params
            ann_u_r = rectify(ann_u, params_n)
            ann_u_rs.append(ann_u_r)
        else:
            ann_u_rs.append(None)

    df['ann_u_r'] = ann_u_rs

In [ ]:
def get_camera_metadata(stereo_parameters):
    camera_metadata = {
        'focalLengthPixel': stereo_parameters['CameraParameters1']['FocalLength'][0],
        'baseline': abs(stereo_parameters['TranslationOfCamera2'][0] / 1e3),
        'focalLength': stereo_parameters['CameraParameters1']['FocalLength'][0] * 3.45e-6,
        'pixelCountWidth': 4096,
        'pixelCountHeight': 3000,
        'imageSensorWidth': 0.01412,
        'imageSensorHeight': 0.01035
    }
    
    return camera_metadata


def add_weights_u_r(df, camera_metadata):
    
    weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    pred_weights = []
    for idx, row in df.iterrows():
        ann = row.ann_u_r
        if ann is not None:
            cm = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, _, _ = weight_estimator.predict(ann, cm)
            pred_weights.append(weight)
        else:
            pred_weights.append(None)
    
    df['weight_u_r'] = pred_weights

In [ ]:
def construct_new_stereo_parameters(stereo_parameters_o, stereo_parameters_n, 
                                    change_PP_1=False, change_PP_2=False, 
                                    change_focal_length_1=False, change_focal_length_2=False,
                                    change_rd_1=False, change_rd_2=False, 
                                    change_R=False, change_T=False):
    
    stereo_parameters_new = dict(stereo_parameters_o)
    
    if change_PP_1:
        stereo_parameters_new['CameraParameters1']['PrincipalPoint'] = \
            stereo_parameters_n['CameraParameters1']['PrincipalPoint']
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][2][:2] = \
            stereo_parameters_n['CameraParameters1']['PrincipalPoint']
    if change_PP_2:
        stereo_parameters_new['CameraParameters2']['PrincipalPoint'] = \
            stereo_parameters_n['CameraParameters2']['PrincipalPoint']
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][2][:2] = \
            stereo_parameters_n['CameraParameters2']['PrincipalPoint']
    if change_focal_length_1:
        stereo_parameters_new['CameraParameters1']['FocalLength'] = \
            stereo_parameters_n['CameraParameters1']['FocalLength']
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][0][0] = \
            stereo_parameters_n['CameraParameters1']['FocalLength'][0]
        stereo_parameters_new['CameraParameters1']['IntrinsicMatrix'][1][1] = \
            stereo_parameters_n['CameraParameters1']['FocalLength'][1]
    if change_focal_length_2:
        stereo_parameters_new['CameraParameters2']['FocalLength'] = \
            stereo_parameters_n['CameraParameters2']['FocalLength']
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][0][0] = \
            stereo_parameters_n['CameraParameters2']['FocalLength'][0]
        stereo_parameters_new['CameraParameters2']['IntrinsicMatrix'][1][1] = \
            stereo_parameters_n['CameraParameters2']['FocalLength'][1]
    if change_rd_1:
        stereo_parameters_new['CameraParameters1']['RadialDistortion'] = \
            stereo_parameters_n['CameraParameters1']['RadialDistortion']
    if change_rd_2:
        stereo_parameters_new['CameraParameters2']['RadialDistortion'] = \
            stereo_parameters_n['CameraParameters2']['RadialDistortion']
    if change_R:
        stereo_parameters_new['RotationOfCamera2'] = stereo_parameters_n['RotationOfCamera2']
    if change_T:
        stereo_parameters_new['TranslationOfCamera2'] = stereo_parameters_n['TranslationOfCamera2']
        
    return stereo_parameters_new

In [ ]:
import math

# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :

    assert(isRotationMatrix(R))
    
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    
    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z]) * 180.0 / np.pi

In [ ]:
stereo_parameter_urls = [
#     's3://aquabyte-stereo-parameters/L40029797_R40020184/2020-10-05T22:39:45.664664000Z_L40029797_R40020184_stereo-parameters.json',
#     's3://aquabyte-stereo-parameters/L40029797_R40020184/2021-02-25T11:30:42.149694000Z_L40029797_R40020184_stereo-parameters.json',
    's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-01-07T12:17:15.547471000Z_L40076048_R40076044_stereo-parameters.json',
    's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-03-10T14:15:41.082425000Z_L40076048_R40076044_stereo-parameters.json'
]

# stereo_parameter_urls = [
#     'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-31T13%3A58%3A30Z-pfe-1424520021297-b0d-452d/cal_output/2021-03-31T14-55-13.320040000Z/stereo_params.json',
#     'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-31T14%3A13%3A37Z-pfe-1424520021297-19a-4b84/cal_output/2021-03-31T14-39-29.128047000Z/stereo_params.json',
#     'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-31T14%3A19%3A00Z-pfe-1424520021297-f19-4f59/cal_output/2021-03-31T15-10-06.143295000Z/stereo_params.json',
#     'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-31T14%3A24%3A59Z-pfe-1424520021297-7b3-4f03/cal_output/2021-03-31T15-25-29.102021000Z/stereo_params.json',
#     'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-31T14%3A30%3A52Z-pfe-1424520021297-179-4f49/cal_output/2021-03-31T15-41-12.727351000Z/stereo_params.json'
# ]

# stereo_parameter_urls = [x.replace('%3A', ':') for x in stereo_parameter_urls]


analysis_data = defaultdict(list)
for stereo_parameter_url in stereo_parameter_urls:
    if stereo_parameter_url.startswith('s3://'):
        stereo_parameters_components = stereo_parameter_url.replace('s3://', '').split('/')
        bucket = stereo_parameters_components[0]
        key = os.path.join(*stereo_parameters_components[1:])
        stereo_parameters_f = s3.download_from_s3(bucket, key)
    elif stereo_parameter_url.startswith('https://'):
        stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameter_url)
    else:
        raise Exception('Unrecognized URL format')
    stereo_params = json.load(open(stereo_parameters_f))
    
    pp_1 = stereo_params['CameraParameters1']['PrincipalPoint']
    pp_2 = stereo_params['CameraParameters2']['PrincipalPoint']
    fl_1 = stereo_params['CameraParameters1']['FocalLength']
    fl_2 = stereo_params['CameraParameters2']['FocalLength']
    rd_1 = stereo_params['CameraParameters1']['RadialDistortion']
    rd_2 = stereo_params['CameraParameters2']['RadialDistortion']
    td_1 = stereo_params['CameraParameters1']['TangentialDistortion']
    td_2 = stereo_params['CameraParameters2']['TangentialDistortion']
    R = stereo_params['RotationOfCamera2']
    R_euler = list(rotationMatrixToEulerAngles(np.array(R)))
    T = stereo_params['TranslationOfCamera2']
    
    analysis_data['stereo_paramter_url'].append(stereo_parameter_url)
    analysis_data['pp_1'].append(pp_1)
    analysis_data['pp_2'].append(pp_2)
    analysis_data['fl_1'].append(fl_1)
    analysis_data['fl_2'].append(fl_2)
    analysis_data['rd_1'].append(rd_1)
    analysis_data['rd_2'].append(rd_2)
    analysis_data['td_1'].append(td_1)
    analysis_data['td_2'].append(td_2)
    analysis_data['R'].append(R)
    analysis_data['R_euler'].append(R_euler)
    analysis_data['T'].append(T)
    
    

In [ ]:
df = pd.DataFrame(analysis_data)

In [ ]:
df.R_euler.values

In [ ]:
df.T